全市场主动卖出成交额占比 均值

In [1]:
import polars as pl
import numpy as np

# 读取数据
data = pl.read_parquet("hourly_data.pa")

df = data.sort('open_time').sort('symbol')

df = df.with_columns([
    (pl.col('quote_volume') - pl.col('taker_buy_quote_volume')).alias('diff'),
    (pl.col('close') / (pl.col('close').shift(1).over('symbol')) - 1).alias('return')
])

# 计算每个时刻所有 symbol 的 taker_buy_quote_volume 的总和
df = df.with_columns([
    pl.col("diff").sum().over("open_time").alias("total_diff_volume")
])

# 计算每个 symbol 在每个时刻的 taker_buy_quote_volume 与总和的比例
df = df.with_columns([
    (pl.col("diff") / pl.col("total_diff_volume")).alias("diff_ratio_perday")
])

df

open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,symbol,diff,return,total_diff_volume,diff_ratio_perday
datetime[μs],f64,f64,f64,f64,f64,datetime[μs],f64,i64,f64,f64,str,f64,f64,f64,f64
2022-09-09 13:00:00,0.4728,0.4874,0.3663,0.451,6.5705837e7,2022-09-09 13:59:59.999,2.8487e7,124200,3.3701922e7,1.4632e7,"""1000LUNCUSDT""",1.3855e7,null,1.3956e9,0.009928
2022-09-09 14:00:00,0.451,0.4609,0.4277,0.4592,4.1316126e7,2022-09-09 14:59:59.999,1.8355e7,94104,2.0298138e7,9.0220e6,"""1000LUNCUSDT""",9.3333e6,0.018182,9.4518e8,0.009875
2022-09-09 15:00:00,0.4594,0.4734,0.4416,0.4555,4.4134479e7,2022-09-09 15:59:59.999,2.0198e7,103417,2.2353913e7,1.0239e7,"""1000LUNCUSDT""",9.9586e6,-0.008057,1.1622e9,0.008569
2022-09-09 16:00:00,0.4554,0.458,0.4175,0.4297,4.6242265e7,2022-09-09 16:59:59.999,2.0187e7,84951,2.1467246e7,9.3760e6,"""1000LUNCUSDT""",1.0811e7,-0.056641,1.3236e9,0.008168
2022-09-09 17:00:00,0.4297,0.4418,0.4071,0.4156,2.661109e7,2022-09-09 17:59:59.999,1.1376e7,50896,1.2655405e7,5.4132e6,"""1000LUNCUSDT""",5.9630e6,-0.032814,1.0146e9,0.005877
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022-12-31 19:00:00,0.1494,0.1498,0.1494,0.1496,748908.5,2022-12-31 19:59:59.999,112011.25443,882,481198.1,71987.20389,"""ZRXUSDT""",40024.05054,0.001339,1.4458e8,0.000277
2022-12-31 20:00:00,0.1496,0.1497,0.1494,0.1496,501195.0,2022-12-31 20:59:59.999,74979.34106,751,282290.8,42242.69096,"""ZRXUSDT""",32736.6501,0.0,1.0070e8,0.000325
2022-12-31 21:00:00,0.1496,0.1497,0.149,0.149,413737.4,2022-12-31 21:59:59.999,61817.02306,583,132775.7,19839.82814,"""ZRXUSDT""",41977.19492,-0.004011,1.3657e8,0.000307


In [2]:
# 取均值
df = df.with_columns([
    pl.col("diff_ratio_perday").rolling_mean(28).over("symbol").alias("mean_diff_ratio_perday")
])

factors = df.select(['open_time', 'symbol', 'diff_ratio_perday'])